In [3]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 38.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 60.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 3.7 MB/s eta 0:00:00


In [4]:
!pip install pyngrok
from pyngrok import ngrok

In [22]:
%%writefile app.py
import streamlit as st
import torch
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt

# Assuming CustomModel and Configuration are properly defined
# Make sure to import or define these classes

class Configuration:
    # Prediction Target Settings
    TARGET_SIZE = 1

    # Model Settings
    MODEL = {
        "name": 'Unet',
        "backbone": 'se_resnext50_32x4d',
        "input_channels": 5  # Originally 65
    }

    # Training Configuration
    IMAGE_SIZE = 512
    INPUT_SIZE = 512
    TRAIN_BATCH_SIZE = 16
    VALID_BATCH_SIZE = 32  # Computed as train_batch_size * 2
    EPOCHS = 20
    LEARNING_RATE = 6e-5
    CHOPPING_PERCENTILE = 1e-3

    # Validation Fold
    VALID_ID = 1


    # Training Settings
    TRAINING = {
        "image_size": 512,
        "train_batch_size": 16,
        "valid_batch_size": 32,  # Computed as train_batch_size * 2
        "epochs": 20,
        "learning_rate": 6e-5,
        "chopping_percentile": 1e-3,
        "valid_fold_id": 1
    }

#     # Augmentation Settings
#     AUGMENTATION = {
#         "train": A.Compose([
#             A.Rotate(limit=45, p=0.5),
#             A.RandomScale(scale_limit=(0.8, 1.25), interpolation=cv2.INTER_CUBIC, p=0.5),
#             A.RandomCrop(512, 512, p=1),
#             A.RandomGamma(p=0.75),
#             A.RandomBrightnessContrast(p=0.5),
#             A.GaussianBlur(p=0.5),
#             A.MotionBlur(p=0.5),
#             A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.5),
#             ToTensorV2(transpose_mask=True),
#         ]),
#         "valid": A.Compose([
#             ToTensorV2(transpose_mask=True),
#         ])
#     }
class CustomModel(nn.Module):
    def __init__(self, configuration, weight=None):
        super(CustomModel, self).__init__()
        # Initialize the Unet model with parameters from the configuration
        self.model = smp.Unet(
            encoder_name=configuration.MODEL['backbone'],
            encoder_weights=weight,
            in_channels=configuration.MODEL['input_channels'],
            classes=configuration.TARGET_SIZE,
            activation=None  # Linear activation on output
        )

    def forward(self, x):
        # Forward pass through the model
        output = self.model(x)
        # Reduce the last dimension if necessary and apply sigmoid activation
        return output[:, 0]  # Assuming binary segmentation with a single class output

@st.cache_data()
def load_model():
    configuration = Configuration()
    model = CustomModel(configuration).cuda()
    model_path = "/kaggle/input/my-se-resnext-model/se_resnext50_32x4d_19_loss0.05_score0.89_val_loss0.28_val_score0.80.pt"
    model.load_state_dict(torch.load(model_path, map_location='cuda'))
    model.eval()
    return model

def pad_to_multiple(image, multiple=32):
    height, width = image.shape[1:3]
    pad_height = (multiple - height % multiple) % multiple
    pad_width = (multiple - width % multiple) % multiple
    padding = (0, 0, pad_width // 2, pad_width - pad_width // 2, pad_height // 2, pad_height - pad_height // 2)
    padded_image = F.pad(image, padding, mode='constant', value=0)
    return padded_image, padding

def unpad_from_multiple(padded_image, padding):
    left, right, top, bottom = padding[2], padding[3], padding[4], padding[5]
    return padded_image[..., top:-bottom, left:-right]

def process_image(img):
    img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)
    img = cv2.resize(img, (512, 512))  # Resize to expected input dimensions
    img = np.expand_dims(img, axis=0)  # Add channel dimension
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return torch.tensor(img, dtype=torch.float32)

def perform_inference(model, images):
    model.eval()
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)
    images = images.to(device)
    with torch.no_grad():
        preds = model(images)
    preds = torch.sigmoid(preds.squeeze(1))  # Remove the extra dimension if present
    return (preds > 0.5).cpu()

def visualize_prediction(image, prediction):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    
    ax1.imshow(image, cmap='gray')
    ax1.set_title('Original Image')
    ax1.axis('off')

    ax2.imshow(image, cmap='gray')
    ax2.imshow(prediction, alpha=0.5, cmap='Reds')  # Red mask overlay
    ax2.set_title('Prediction Overlay')
    ax2.axis('off')

    return fig

# Streamlit app
st.title('Blood Vessel Segmentation App')

# File uploader for 5 images
uploaded_files = st.file_uploader("Upload 5 consecutive image slices", type=["png", "jpg", "jpeg", 'tif'], accept_multiple_files=True)

if uploaded_files and len(uploaded_files) == 5:
    images = torch.cat([process_image(Image.open(file)) for file in uploaded_files], dim=1)
    
    model = load_model()
    predictions = perform_inference(model, images)
    
    slice_index = st.slider("Select which slice to visualize", 0, 4, 2)
    
    fig = visualize_prediction(images[:, slice_index], predictions[slice_index])
    st.pyplot(fig)
else:
    st.warning("Please upload exactly 5 image slices.")


Overwriting app.py


In [12]:
!ngrok config add-authtoken 2VJXf1SjlcpG5WPLyrgDcEaM2ct_7KytqnABitp5VfqfxZCYf

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [13]:
!wget -q -O - ipv4.icanhazip.com

34.22.177.194


In [14]:
!npm install -g localtunnel

m##################] / reify:yargs: timing reifyNode:node_modules/localtunnel/nles/lo
changed 22 packages in 838ms

3 packages are looking for funding
  run `npm fund` for details


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://172.19.2.2:8502
  External URL: http://34.22.177.194:8502

your url is: https://better-pandas-appear.loca.lt


In [34]:
import importlib

# List of libraries to import and check versions
libraries = {
    "streamlit": "st",
    "torch": "torch",
    "numpy": "np",
    "opencv-python": "cv2",
    "Pillow": "PIL",
    "matplotlib": "matplotlib"
}

for lib, shortname in libraries.items():
    try:
        # Dynamically import the library
        module = importlib.import_module(shortname)
        # Print the version
        print(f"{lib} version: {module.__version__}")
    except ImportError:
        print(f"{lib} is not installed.")
    except AttributeError:
        print(f"Could not determine version for {lib}.")


streamlit is not installed.
torch version: 2.1.2+cpu
numpy is not installed.
opencv-python version: 4.10.0
Pillow version: 9.5.0
matplotlib version: 3.7.5


In [36]:
import torch

In [37]:
print(torch.__version__)

2.1.2+cpu
